In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
os.chdir('../../Data')

In [3]:
df = pd.read_csv('final_data_oot.csv',header=0)

In [4]:
df.columns

Index(['Date', 'Symbol', 'Series', 'Prev Close', 'Open', 'High', 'Low', 'Last',
       'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume',
       '%Deliverble', 'Sector', 'Open_nifty', 'High_nifty', 'Low_nifty',
       'Close_nifty', 'Volume_nifty', 'Turnover_nifty', 'Open_auto',
       'High_auto', 'Low_auto', 'Close_auto', 'Volume_auto', 'Turnover_auto',
       'Open_bank', 'High_bank', 'Low_bank', 'Close_bank', 'Volume_bank',
       'Turnover_bank', 'Open_fmcg', 'High_fmcg', 'Low_fmcg', 'Close_fmcg',
       'Volume_fmcg', 'Turnover_fmcg', 'Open_it', 'High_it', 'Low_it',
       'Close_it', 'Volume_it', 'Turnover_it', 'Open_media', 'High_media',
       'Low_media', 'Close_media', 'Volume_media', 'Turnover_media',
       'Open_metal', 'High_metal', 'Low_metal', 'Close_metal', 'Volume_metal',
       'Turnover_metal', 'Open_pharma', 'High_pharma', 'Low_pharma',
       'Close_pharma', 'Volume_pharma', 'Turnover_pharma', 'Open_realty',
       'High_realty', 'Low_realty',

In [5]:
rename_dict = {col : '_'.join(col.split(' ')) for col in df.columns}

In [6]:
df.rename(rename_dict,copy=False,inplace=True,axis=1)

In [7]:
companies = list(df['Symbol'].unique())

In [8]:
cols_to_shift = [col for col in df.columns if col not in ['Date','Symbol','Series','Prev_Close','Open','Close','Sector']]

In [9]:
rename_dict2 = {col : col+'_pct_change' for col in cols_to_shift}

In [10]:
rename_dict2

{'High': 'High_pct_change',
 'Low': 'Low_pct_change',
 'Last': 'Last_pct_change',
 'VWAP': 'VWAP_pct_change',
 'Volume': 'Volume_pct_change',
 'Turnover': 'Turnover_pct_change',
 'Trades': 'Trades_pct_change',
 'Deliverable_Volume': 'Deliverable_Volume_pct_change',
 '%Deliverble': '%Deliverble_pct_change',
 'Open_nifty': 'Open_nifty_pct_change',
 'High_nifty': 'High_nifty_pct_change',
 'Low_nifty': 'Low_nifty_pct_change',
 'Close_nifty': 'Close_nifty_pct_change',
 'Volume_nifty': 'Volume_nifty_pct_change',
 'Turnover_nifty': 'Turnover_nifty_pct_change',
 'Open_auto': 'Open_auto_pct_change',
 'High_auto': 'High_auto_pct_change',
 'Low_auto': 'Low_auto_pct_change',
 'Close_auto': 'Close_auto_pct_change',
 'Volume_auto': 'Volume_auto_pct_change',
 'Turnover_auto': 'Turnover_auto_pct_change',
 'Open_bank': 'Open_bank_pct_change',
 'High_bank': 'High_bank_pct_change',
 'Low_bank': 'Low_bank_pct_change',
 'Close_bank': 'Close_bank_pct_change',
 'Volume_bank': 'Volume_bank_pct_change',
 'Turn

In [11]:
development = pd.DataFrame()
for company in companies:
    print('development data for {}'.format(company))
    df_append = df[df['Symbol']==company]
    df_append['Date'] = pd.to_datetime(df['Date'])
    df_append.sort_values(by='Date',inplace=True)
    df_append['Prev_Close_max'] = df_append['Prev_Close'].rolling(5).max()
    df_append['Prev_Close_min'] = df_append['Prev_Close'].rolling(5).min()
    df_append['Prev_Close_max_ratio'] = df_append['Prev_Close']/df_append['Prev_Close_max']
    df_append['Prev_Close_min_ratio'] = df_append['Prev_Close_min']/df_append['Prev_Close']
    for index in ['nifty','auto','bank','fmcg','it','media','metal','pharma','realty','psu','500']:
        for i in range(1,6):
            df_append['return_{j}d_{index}'.format(j=i,index=index)] = (df_append['Close_{}'.format(index)].shift(i)-df_append['Open_{}'.format(index)].shift(i))/df_append['Open_{}'.format(index)].shift(i)
        df_append['Voltality_{}'.format(index)] = df_append[['return_{i}d_{index}'.format(i=i,index=index) for i in range(1,6)]].apply(lambda x:np.std(x),axis=1)
    for i in range(1,6):
        df_append['return_{j}d'.format(j=i)] = (df_append['Close'].shift(i)-df_append['Open'].shift(i))/df_append['Open'].shift(i)
    df_append['Voltality'] = df_append[['return_1d','return_2d','return_3d','return_4d','return_5d']].apply(lambda x:np.std(x),axis=1)
    df_append['Open_pct_change'] = df_append['Open'].pct_change().shift(1)
    df_append['Prev_Close_pct_change'] = df_append['Prev_Close'].pct_change()
    for col in cols_to_shift:
        df_append[col] = df_append[col].pct_change().shift(1)
    df_append.rename(rename_dict2,copy=False,inplace=True,axis=1)
    development = development.append(df_append.iloc[5:])
development['actual_return'] = (development['Close']-development['Open'])/development['Open']

development data for ADANIPORTS


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\DELL\Anacond

development data for ASIANPAINT
development data for AXISBANK
development data for BAJAJ-AUTO
development data for BAJFINANCE
development data for BAJAJFINSV
development data for BHARTIARTL
development data for INFRATEL
development data for BPCL
development data for BRITANNIA
development data for CIPLA
development data for COALINDIA
development data for DRREDDY
development data for EICHERMOT
development data for GAIL
development data for GRASIM
development data for HCLTECH
development data for HDFC
development data for HDFCBANK
development data for HEROMOTOCO
development data for HINDALCO
development data for HINDUNILVR
development data for ICICIBANK
development data for IBULHSGFIN
development data for INDUSINDBK
development data for INFY
development data for IOC
development data for ITC
development data for JSWSTEEL
development data for KOTAKBANK
development data for LT
development data for M&M
development data for MARUTI
development data for NTPC
development data for ONGC
development

In [12]:
development.isnull().any()

Date                             False
Symbol                           False
Series                           False
Prev_Close                       False
Open                             False
High_pct_change                  False
Low_pct_change                   False
Last_pct_change                  False
Close                            False
VWAP_pct_change                  False
Volume_pct_change                False
Turnover_pct_change              False
Trades_pct_change                False
Deliverable_Volume_pct_change    False
%Deliverble_pct_change           False
Sector                           False
Open_nifty_pct_change            False
High_nifty_pct_change            False
Low_nifty_pct_change             False
Close_nifty_pct_change           False
Volume_nifty_pct_change          False
Turnover_nifty_pct_change        False
Open_auto_pct_change             False
High_auto_pct_change             False
Low_auto_pct_change              False
Close_auto_pct_change    

In [13]:
development.shape

(9150, 161)

In [14]:
development.columns

Index(['Date', 'Symbol', 'Series', 'Prev_Close', 'Open', 'High_pct_change',
       'Low_pct_change', 'Last_pct_change', 'Close', 'VWAP_pct_change',
       ...
       'Voltality_500', 'return_1d', 'return_2d', 'return_3d', 'return_4d',
       'return_5d', 'Voltality', 'Open_pct_change', 'Prev_Close_pct_change',
       'actual_return'],
      dtype='object', length=161)

In [15]:
null_cols = []
for col in development.columns:
    if development[col].isnull().any():
        print(col)
        null_cols.append(col)

In [16]:
null_cols

[]

In [17]:
development.drop(null_cols,axis=1,inplace=True)

In [18]:
for col in development.columns:
    if development[col].isnull().any():
        print(col)

In [19]:
training = development.loc[development['Date']< '2019-01-01']
testing =  development.loc[development['Date']>='2019-01-01']

In [19]:
os.chdir('../')

In [21]:
training.shape

(85799, 161)

In [20]:
os.chdir('Price Prediction/Input_Data/')

In [21]:
development.to_csv('oot.csv',index=False)

In [23]:
training.to_csv('train.csv',index=False)

In [24]:
testing.to_csv('validation.csv',index=False)

In [22]:
development.head()

,Date,Symbol,Series,Prev_Close,Open,High_pct_change,Low_pct_change,Last_pct_change,Close,VWAP_pct_change,...,Voltality_500,return_1d,return_2d,return_3d,return_4d,return_5d,Voltality,Open_pct_change,Prev_Close_pct_change,actual_return
5,2019-04-01,ADANIPORTS,EQ,378.15,380.95,0.008493,0.025116,-0.003150,383.05,0.005031,...,0.005905,-0.013436,0.033754,-0.002451,0.020064,0.006826,0.016579,0.039176,-0.008261,0.005513
6,2019-04-02,ADANIPORTS,EQ,383.05,383.00,0.001555,0.008788,0.004608,379.70,0.008379,...,0.005356,0.005513,-0.013436,0.033754,-0.002451,0.020064,0.016621,-0.006131,0.012958,-0.008616
7,2019-04-03,ADANIPORTS,EQ,379.70,380.30,-0.001294,-0.003432,-0.001966,382.00,-0.004468,...,0.004619,-0.008616,0.005513,-0.013436,0.033754,-0.002451,0.016655,0.005381,-0.008746,0.004470
8,2019-04-04,ADANIPORTS,EQ,382.00,382.00,0.000777,0.006623,0.000525,380.20,0.005879,...,0.005579,0.004470,-0.008616,0.005513,-0.013436,0.033754,0.016434,-0.007050,0.006057,-0.004712
9,2019-04-05,ADANIPORTS,EQ,380.20,379.00,-0.011002,-0.013947,-0.005512,387.15,-0.011194,...,0.003662,-0.004712,0.004470,-0.008616,0.005513,-0.013436,0.007362,0.004470,-0.004712,0.021504


In [23]:
for col in [col for col in development.columns if col not in ['Date','Symbol','Series','Prev_Close','Open','Close','Sector']]:
    print("{col} : min = {min}, max = {max}".format(col=col,min=development[col].min(),max=development[col].max()))

High_pct_change : min = -0.5022592900658679, max = 0.3238015138772077
Low_pct_change : min = -0.5099193006052454, max = 0.21379310344827607
Last_pct_change : min = -0.5120860927152318, max = 0.3276397515527949
VWAP_pct_change : min = -0.5077907439331654, max = 0.17541018998272895
Volume_pct_change : min = -0.9828846420772573, max = 28.40670984726457
Turnover_pct_change : min = -0.9828333371058683, max = 26.512125021981294
Trades_pct_change : min = -0.9698879272524229, max = 31.66101694915254
Deliverable_Volume_pct_change : min = -0.9935117281888439, max = 120.36554135956418
%Deliverble_pct_change : min = -0.8138864880437914, max = 5.687065368567454
Open_nifty_pct_change : min = -0.020288180520627974, max = 0.07405925484795484
High_nifty_pct_change : min = -0.020806934499392304, max = 0.04948733080072287
Low_nifty_pct_change : min = -0.023275328660185335, max = 0.07299130109437857
Close_nifty_pct_change : min = -0.021382337875651358, max = 0.053191091846648275
Volume_nifty_pct_change : 